In [1]:
import numpy as np

import torch
from torch.utils.data import Dataset
from nnAudio.Spectrogram import CQT1992v2

import pandas as pd

In [2]:
df = pd.read_csv("../input/train_folds.csv")
df.head(3)

,Unnamed: 0,id,target,file_path,fold
0,0,00000e74ad,1,input/train/0/0/0/00000e74ad.npy,2
1,1,00001f4945,0,input/train/0/0/0/00001f4945.npy,1
2,2,0000661522,0,input/train/0/0/0/0000661522.npy,3


In [ ]:

class TrainDataset(Dataset):
    def __init__(self, CFG, df, transform=None):
        self.df = df
        self.file_names = df["file_path"].values
        self.labels = df["target"].values
        # TRYME worth playing with these values
        self.wave_transform = CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=16)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def apply_qtransform(self, waves, transform):
        waves = np.hstack(waves)
        # TRYME maybe worth playing with this scaling
        waves = waves / np.max(waves)
        waves = torch.from_numpy(waves).float()
        image = transform(waves)
        return image

    def __getitem__(self, idx):
        file_path = self.file_names[idx]
        waves = np.load(file_path)
        image = self.apply_qtransform(waves, self.wave_transform)
        if self.transform:
            image = image.squeeze().numpy()
            image = self.transform(image=image)["image"]
        label = torch.tensor(self.labels[idx]).float()
        return image, label

In [4]:
from tqdm import tqdm

In [8]:
pths = df.file_path.values
idxs = df.id.values

wave_transform = CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=16)

def apply_qtransform(waves, transform):
    waves = np.hstack(waves)
    # TRYME maybe worth playing with this scaling
    waves = waves / np.max(waves)
    waves = torch.from_numpy(waves).float()
    image = transform(waves)
    return image

for idx, pth in tqdm(zip(idxs, pths)):
    waves = np.load(f"../{pth}")
    img = apply_qtransform(waves, wave_transform)
    np.save(f"../input/train_1d/{idx}.npy", img)

CQT kernels created, time used = 0.0134 seconds


560000it [1:20:37, 115.75it/s]


In [3]:
!mkdir ../input/train_1d